FORTRESS

[algospot의 FORTRESS 링크](https://algospot.com/judge/problem/read/FORTRESS)

![](http://algospot.com/media/judge-attachments/6b98991b489acef77ed1b63dc31bc32f/castle.svg.png)  
중세의 성과 요새들은 보안을 튼튼히 하면서도 더 넓은 영역을 보호하기 위해 여러 개의 성벽을 갖고 있었다고 하지요. 전세계에서 가장 편집증이 심한 영주가 지은 스트로고(Strawgoh) 요새는 이의 극치를 보여줍니다. 이 요새는 그림과 같이 커다란 원형 외벽 내에 여러 개의 원형 성벽이 겹겹이 지어진 형태로 구성되어 있는데, 어떤 성벽에도 문이 없어서 성벽을 지나가려면 사다리를 타고 성벽을 오르내려야 합니다. 요새 내에서도 한 곳에서 다른 곳으로 이동하는 데 시간이 너무 오래 걸린다는 원성이 자자해지자, 영주는 요새 내에서 왕래가 불편한 곳들을 연결하는 터널을 만들기로 했습니다. 계획을 세우기 위해 요새 내에서 서로 왕래하기 위해 가장 성벽을 많이 넘어야 하는 두 지점을 찾으려고 합니다. 예를 들어 위 그림의 경우, 별표로 표시된 두 지점 간을 이동하기 위해서는 다섯 번이나 성벽을 넘어야 하지요.

성벽들의 정보가 주어질 때 가장 성벽을 많이 넘어야 하는 두 지점 간을 이동하기 위해 몇 번이나 성벽을 넘어야 하는지 계산하는 프로그램을 작성하세요.

solution함수의  
walls : 벽의 [중심 x좌표, 중심 y좌표, 반지름] 정보를 주는 리스트  

In [1]:
import math

class Tree(): # Tree 클래스
    index = None # node 인덱스
    wall_info = None # [중심 x좌표, 중심 y좌표, 반지름] 벽의 정보
    parent = None # 노드의 부모
    children = None # 노드의 자식노드들
    
    def __init__(self, wall_info, index=0, parent=None): # 클래스 생성자
        self.wall_info = wall_info
        self.index = index
        self.parent = parent
        self.children = []
        
    def add_child(self, child): # 자식 노드 추가
        add = True
        for i in self.children:
            if i.index == child.index:
                add = False
                break
        if add:
            self.children.append(child)
    
    def delete_child(self, child): # 자식 노드 삭제
        for i in self.children:
            if i.index == child.index:
                self.children.remove(i)
                break
    
    def add_parent(self, parent): # 부모 노드 추가
        if self.parent is None:
            self.parent = parent     
        
    def __repr__(self): # 인스턴스 출력 방법
        return "No."+ str(self.index) + " Node"
    
def enclose(nodes, a_idx, b_idx): # 노드 a 가 노드 b를 포함하는지
    x_a, y_a, r_a = nodes[a_idx].wall_info[0], nodes[a_idx].wall_info[1], nodes[a_idx].wall_info[2]
    x_b, y_b, r_b = nodes[b_idx].wall_info[0], nodes[b_idx].wall_info[1], nodes[b_idx].wall_info[2]
    return r_a > r_b and pow(y_a - y_b, 2) + pow(x_a - x_b, 2) < pow(r_a - r_b, 2)
    # r_a > r_b 이고 (y_a - y_b)^2 + (x_a - x_b)^2 < (r_a - r_b)^2 를 만족할 때
    # a 구역은 b 구역을 완전히 포함한다.
    
def is_child(a_idx, b_idx, nodes, memo): # node a가 node b의 부모인지
    if memo[a_idx][b_idx] is not None: # 메모이재이션
        return memo[a_idx][b_idx]
    
    if not enclose(nodes, a_idx, b_idx): # 만약 node a가 node b를 포함하지 않는다면
        memo[a_idx][b_idx] = False # 이들은 부모자식 관계가 아니다.
        return memo[a_idx][b_idx]
    
    memo[a_idx][b_idx] = True
    for n in nodes:
        # node a, node b를 제외한 다른 노드들과의 포함관계가 중간에 껴있는지 확인
        if n != nodes[a_idx] and n != nodes[b_idx] and enclose(nodes, a_idx, n.index) and enclose(nodes, n.index, b_idx):
            memo[a_idx][b_idx] = False
            break  # 중간에 껴있는 구역이 있다면 직접적인 부모자식관계가 아니다.

    if memo[a_idx][b_idx]: # 만약 부모자식 관계라면
        nodes[a_idx].add_child(nodes[b_idx]) # node a의 자식에 node b를 추가
        nodes[b_idx].add_parent(nodes[a_idx]) # node b의 부모에 node a를 추가
    else: # 부모자식 관계가 아니라면(이 경우는 직접적인 부모자식 관계를 뜻함)
        nodes[a_idx].delete_child(nodes[b_idx]) 
        # 만약 부모자식 관계라고 설정해둔 것을 삭제 
    return memo[a_idx][b_idx] 

def find_longest(root): # root로 부터의 length를 구하는 함수
    h = 0
    if root.children == []: # 만약 자식이 없다면 0을 반환
        return 0
    for i in root.children:
        h = max(h, find_longest(i)+1) # 모든 자식들의 서브트리를 순회하며 가장 긴 length를 찾는다.
    return h
    
def solution(walls):
    nodes = []
    for i in range(len(walls)): # 벽에 대한 정보를 가지는 노드들을 생성한다.
        nodes.append(Tree(walls[i], i))
        
    memo = [[None for _ in range(len(nodes))]for _ in range(len(nodes))] 
    # 직접 부모자식관계 여부를 저장해둘 메모이재이션 매트릭스
    for i in range(len(nodes)-1):
        for j in range(i+1, len(nodes)):
            is_child(i, j, nodes, memo)
            # 직접적인 부모자식관계인지 모두 확인한다.
    depths = [] # root에서 leaf까지의 모든 depth를 구할 것이다.
    for i in nodes[0].children:
        depths.append(find_longest(i)+1)
    return sum(sorted(depths, reverse=True)[:2])
    # 구한 depth 중에서 가장 긴 depth 2개를 합친 값을 반환한다.


In [2]:
walls = [[5, 5, 15],
         [5, 5, 10],
         [5, 5, 5]]

In [3]:
solution(walls)

2

In [4]:
walls = [[21, 15, 20],
         [15, 15, 10],
         [13, 12, 5],
         [12, 12, 3],
         [19, 19, 2],
         [30, 24, 5],
         [32, 10, 7],
         [32, 9, 4]]

In [5]:
solution(walls)

5